In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.linear_model import LogisticRegression
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


from sklearn.impute import SimpleImputer
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/toxic-comment-model/doc2vec-toxic.model.docvecs.vectors_docs.npy
/kaggle/input/toxic-comment-model/doc2vec-toxic.model
/kaggle/input/test-features/Test_features_toxic_comment.npy


In [2]:
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip

Archive:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
  inflating: train.csv               
Archive:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
  inflating: test.csv                
Archive:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
  inflating: test_labels.csv         


In [3]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
submission=pd.read_csv('sample_submission.csv')
test_labels=pd.read_csv('test_labels.csv')

In [4]:
train_df.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [5]:
test_df

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


### So we can see that a comment can have multiple labels

In [6]:
cols_to_drop=['id']
train_df.drop(cols_to_drop,axis=1,inplace=True)


# Preprocessing Training Data

pre-process each line (tokenize text into individual words, remove punctuation, set to lowercase, etc)

In [7]:
# There might be some missing values so we have to handle them as always
COMMENT = 'comment_text'
train_df[COMMENT].fillna("unknown", inplace=True)
test_df[COMMENT].fillna("unknown", inplace=True)


In [8]:
import gensim

def tokeniser_gen(df,tokens_only=False):
   
    for i,iterator in enumerate(range(len(df))):
        line=df.loc[iterator]['comment_text']
        tokens=gensim.utils.simple_preprocess(line)
        if tokens_only:
            yield tokens
        else:
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

        
train_corpus=list(tokeniser_gen(train_df))

test_corpus = list(tokeniser_gen(test_df, tokens_only=True))


In [9]:
# print(len(train_corpus))
# print(train_corpus[52442][0])
# # train_df.loc[train_df['comment_text'].str.len()==6]

In [10]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=50)

## Build Vocabulary
Essentially, the vocabulary is a dictionary (accessible via model.wv.vocab) of all of the unique words extracted from the training corpus along with the count 

(e.g., model.wv.vocab['penalty'].count for counts for the word penalty).


In [11]:
model.build_vocab(train_corpus)

In [12]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [13]:
model.save("doc2vec-toxic.model")

In [14]:
from gensim.models.doc2vec import Doc2Vec

model_df = Doc2Vec.load("/kaggle/input/toxic-comment-model/doc2vec-toxic.model")
vector = model_df.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
print(vector)

[-3.00585449e-01 -3.12538326e-01 -3.93327653e-01  2.58546472e-01
  2.70646900e-01 -4.80407119e-01 -6.27700210e-01 -9.66509163e-01
 -7.38039389e-02 -5.44448614e-01  9.09341797e-02  1.86731651e-01
  1.64255928e-02  4.68516380e-01 -5.44621944e-01 -2.87976950e-01
  2.09801391e-01 -4.45243299e-01  3.34608592e-02 -2.05966488e-01
  3.86952370e-01 -9.72304717e-02 -2.78474063e-01  2.50923455e-01
 -2.65111148e-01  1.37147769e-01  7.45327115e-01  7.01007366e-01
  4.53064352e-01  4.61646259e-01  8.05839181e-01  4.73835915e-01
 -2.65400827e-01  1.62150338e-01 -3.49679530e-01  2.00332850e-02
 -1.14737742e-01 -1.65716469e-01  1.79438487e-01 -1.49890155e-01
  7.37734884e-02  3.10572416e-01  2.54594833e-01  4.37777668e-01
  1.66243613e-02  1.86889052e-01 -5.99311888e-01 -6.10400617e-01
 -7.21803904e-01  7.23093808e-01  2.12422851e-02 -5.63834429e-01
 -5.68385124e-01 -3.30766112e-01 -1.32906916e-05  5.65716624e-01
 -5.27322888e-02 -7.19054580e-01  1.23337910e-01  2.28112638e-01
  3.02164346e-01 -4.71858

In [1]:
# Please uncomment this when you try to run this code



# empty_test_fea=[]
# for i in range(len(test_corpus[:])):
#     vector=model_df.infer_vector(test_corpus[i])
#     empty_test_fea.append(vector)
#     print(i)

In [16]:
test_features=empty_test_fea[:]
test_features=np.array(test_features)
test_features.shape
np.save('Test_features_toxic_comment',test_features)


### Loading training and testing Features

In [17]:
train_features=np.load('/kaggle/input/toxic-comment-model/doc2vec-toxic.model.docvecs.vectors_docs.npy')
test_features=np.load('/kaggle/input/test-features/Test_features_toxic_comment.npy')
print(test_features.shape)
print(train_features.shape)

(153164, 100)
(159571, 100)


Basic Naive Bayes Equation

In [18]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
#     print(p)
    return (p+1) / ((y==y_i).sum()+1)

In [19]:
x = train_features
test_x = test_features

In [20]:
from scipy import sparse
x = sparse.csr_matrix(x)
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(x)


SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='mean', verbose=0)

Fit a model for one dependent at a time:

In [21]:
def get_mdl(y):
    y = y.values
    r=pr(1,y) / pr(0,y)
#     r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [22]:
from sklearn.linear_model import LogisticRegression
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
preds = np.zeros((len(test_df), len(label_cols)))



for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(train_df[j])
    preds[:,i] = m.predict_proba(np.multiply(test_x,r))[:,1]

fit toxic


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


fit severe_toxic


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


fit obscene


...

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


fit insult


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


fit identity_hate


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [23]:
submid = pd.DataFrame({'id': submission["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
submission.to_csv('submission.csv', index=False)